In [1]:
import tensorflow as tf
import numpy as np
import uuid
import sys
import scipy.stats 
import matplotlib
import itertools 
matplotlib.use('Agg')
import matplotlib.pyplot as plt

np.random.seed(0)
tf.set_random_seed(0)

In [2]:
def neural_net(params={}, m=1, mode='train', verbose=False):
    '''m: Dimension of x0 (default =1)
       
       params: parameter dictionary that contains
       num_batches: number of batches 
       batch_size: number of samples per batch
       sigma_x0: standard deviation of x0
       sigma_z: standard deviation of noise
       optimizer: the optimizer used to minimize cost
    '''
    #Placeholders for inputs
    x0 = tf.placeholder(tf.float32, [None, m])
    z = tf.placeholder(tf.float32, [None, m])


    #The layers
   
    x1 = tf.layers.dense(inputs=x0, units=1, activation=None , use_bias=True, name = 'layer1')  #Linear activation
    y1 = x1 + z
    
    with tf.variable_scope('layer1', reuse=True):
        w1 = tf.get_variable('kernel')
        b1 = tf.get_variable('bias')
    
    #Define loss function    
    cost = tf.reduce_mean(tf.reduce_mean((y1-1)**2,axis=1))

    #Learning rate and optimizer
    optimizer_function = params['optimizer_function']
    learning_rate = params['learning_rate']
    adaptive_learning_rate = tf.placeholder_with_default(learning_rate, [])
    optimizer = optimizer_function(adaptive_learning_rate).minimize(cost)
            
   
    tf.summary.scalar("Cost", cost)
    tf.summary.scalar("w1", tf.norm(w1))
    tf.summary.scalar("b1", tf.norm(b1))
    merged_summary_op = tf.summary.merge_all()   
    
    with tf.Session() as sess:
            uniq_id = "/tmp/tensorboard-layers-api/basic2" + uuid.uuid1().__str__()[:6]
            summary_writer = tf.summary.FileWriter(uniq_id, graph=tf.get_default_graph())
            
            if verbose is True:
                print("Training...")
           
        
            init_op = tf.global_variables_initializer()
            sess.run(init_op) 
            
            batch_size = params['batch_size']
            sigma_x0 = params['sigma_x0']
            sigma_z = params['sigma_z']
            max_epochs = params['max_epochs']
            
            
           
            '''Train the nn on training set constructed by sampling x0 and z independently from gaussian distributions\
            with means 0 and standard deviations sigma_x0 and sigma_z respectively'''
            #WARNING: Can only be used when marginals along dimensions are independent 
            train_costs = np.zeros((max_epochs,1))
            for i in range(max_epochs): #Splitting into multiple batches so as to not run into memory issues
                batch_x0 = np.random.normal(loc=np.zeros((m,)), scale = sigma_x0, size=[batch_size,m])
                batch_z = np.random.normal(loc=np.zeros((m,)), scale = sigma_z, size=[batch_size,m])

                _,w1_tmp,b1_tmp,batch_cost,summary = sess.run([optimizer,w1,b1,cost,merged_summary_op], feed_dict = {x0:batch_x0, z:batch_z})
                train_costs[i] = batch_cost
                summary_writer.add_summary(summary, i)
           
              
                if verbose is True:
                    print("---Epoch: {}, Cost: {}".format(i,batch_cost))
#             return train_costs

#         if mode == 'evaluate_sampled':
            if verbose is True:
                print("Evaluating on sampled gaussians...")
            num_batches = params['num_batches']
            batch_size = params['batch_size']
            sigma_x0 = params['sigma_x0']
            sigma_z = params['sigma_z']
            '''Evaluate the nn on test set constructed by sampling x0 and z independently from gaussian distributions\
            with means 0 and standard deviations sigma_x0 and sigma_z respectively'''
            #WARNING: Can only be used when marginals along dimensions are independent 
            test_costs = np.zeros((num_batches,1))
            for i in range(num_batches): #Splitting into multiple batches so as to not run into memory issues
                batch_x0 = np.random.normal(loc=np.zeros((m,)), scale = sigma_x0, size=[batch_size,m])
                batch_z = np.random.normal(loc=np.zeros((m,)), scale = sigma_z, size=[batch_size,m])

                batch_cost = sess.run(cost, feed_dict = {x0:batch_x0, z:batch_z})
                test_costs[i] = batch_cost
                if verbose is True:
                    print("---Batch: {}, Cost: {}".format(i,batch_cost))

            avg_test_cost = float(np.mean(test_costs, axis = 0))
            if verbose is True:
                print("Average test cost: {}".format(avg_test_cost))
            return avg_test_cost


#Evaluate on sampled gaussians
params = {'num_batches':10, 'batch_size':100000, 'sigma_x0':2, 'sigma_z':1,\
          'optimizer_function':tf.train.AdamOptimizer, 'learning_rate':1, 'max_epochs':500}

neural_net(params=params, mode = 'train', verbose = True)
# neural_net(params=params, mode = 'evaluate_sampled', verbose = True)
        

Training...
---Epoch: 0, Cost: 10.662944793701172
---Epoch: 1, Cost: 1.9072867631912231
---Epoch: 2, Cost: 2.05507493019104
---Epoch: 3, Cost: 4.6709303855896
---Epoch: 4, Cost: 4.890200138092041
---Epoch: 5, Cost: 3.2700562477111816
---Epoch: 6, Cost: 1.7496429681777954
---Epoch: 7, Cost: 1.2418804168701172
---Epoch: 8, Cost: 1.7021156549453735
---Epoch: 9, Cost: 2.397653818130493
---Epoch: 10, Cost: 2.688009023666382
---Epoch: 11, Cost: 2.3686726093292236
---Epoch: 12, Cost: 1.7773725986480713
---Epoch: 13, Cost: 1.3059738874435425
---Epoch: 14, Cost: 1.1445101499557495
---Epoch: 15, Cost: 1.3295968770980835
---Epoch: 16, Cost: 1.6315572261810303
---Epoch: 17, Cost: 1.8079979419708252
---Epoch: 18, Cost: 1.6949056386947632
---Epoch: 19, Cost: 1.4120787382125854
---Epoch: 20, Cost: 1.1458392143249512
---Epoch: 21, Cost: 1.0380823612213135
---Epoch: 22, Cost: 1.1386785507202148
---Epoch: 23, Cost: 1.3389225006103516
---Epoch: 24, Cost: 1.4194092750549316
---Epoch: 25, Cost: 1.367357969

---Epoch: 209, Cost: 0.9993327856063843
---Epoch: 210, Cost: 1.0027765035629272
---Epoch: 211, Cost: 1.0009390115737915
---Epoch: 212, Cost: 1.0032278299331665
---Epoch: 213, Cost: 1.0035709142684937
---Epoch: 214, Cost: 1.004224181175232
---Epoch: 215, Cost: 0.9951908588409424
---Epoch: 216, Cost: 0.9980842471122742
---Epoch: 217, Cost: 1.001580834388733
---Epoch: 218, Cost: 1.0002614259719849
---Epoch: 219, Cost: 0.9982234239578247
---Epoch: 220, Cost: 0.999026894569397
---Epoch: 221, Cost: 1.005328893661499
---Epoch: 222, Cost: 0.996045708656311
---Epoch: 223, Cost: 1.0012542009353638
---Epoch: 224, Cost: 0.9980506300926208
---Epoch: 225, Cost: 0.998698353767395
---Epoch: 226, Cost: 1.000641107559204
---Epoch: 227, Cost: 0.9963085651397705
---Epoch: 228, Cost: 0.9917452931404114
---Epoch: 229, Cost: 0.9984707236289978
---Epoch: 230, Cost: 0.9991679787635803
---Epoch: 231, Cost: 0.9930846095085144
---Epoch: 232, Cost: 0.9985942244529724
---Epoch: 233, Cost: 1.0009957551956177
---Epoc

---Epoch: 420, Cost: 0.9955219030380249
---Epoch: 421, Cost: 1.006416916847229
---Epoch: 422, Cost: 1.003395676612854
---Epoch: 423, Cost: 0.9960865378379822
---Epoch: 424, Cost: 1.003659725189209
---Epoch: 425, Cost: 1.002113699913025
---Epoch: 426, Cost: 0.9989617466926575
---Epoch: 427, Cost: 1.0020192861557007
---Epoch: 428, Cost: 1.0022387504577637
---Epoch: 429, Cost: 1.003105640411377
---Epoch: 430, Cost: 0.9993565678596497
---Epoch: 431, Cost: 0.9936535358428955
---Epoch: 432, Cost: 0.9910346269607544
---Epoch: 433, Cost: 1.0010002851486206
---Epoch: 434, Cost: 0.9973610639572144
---Epoch: 435, Cost: 1.0066142082214355
---Epoch: 436, Cost: 1.006606101989746
---Epoch: 437, Cost: 0.994981586933136
---Epoch: 438, Cost: 0.9986860156059265
---Epoch: 439, Cost: 0.994005560874939
---Epoch: 440, Cost: 1.0052363872528076
---Epoch: 441, Cost: 0.9953849911689758
---Epoch: 442, Cost: 0.9964863061904907
---Epoch: 443, Cost: 0.9992861151695251
---Epoch: 444, Cost: 1.0032850503921509
---Epoch

1.0004443228244781

In [3]:
 # Test over a continuous range of X
    
num_x0_points = 10
x_stddev = 1
x0_test = np.linspace(-3 * x_stddev, 3 * x_stddev, num=num_x0_points)  
x0_distribution = scipy.stats.norm(loc=0.0, scale=x_stddev)
print(x0_test)
print(x0_distribution.pdf(x0_test))
#       total_density = 0.0
#       for i in range(num_x0_points):
#           u1t, u2t, wits_cost_t  = 0, 0, 0
          

#           for _ in range(test_averaging):
#             u1tmp, u2tmp, wits_cost_tmp = sess.run(
#                   [u1, u2, wits_cost],
#                   feed_dict={x0: x0_test[i].reshape((1, 1)), x1: x1_test[i].reshape((1, 1)),
#                   z: z_test[i].reshape((1, 1)), y2: y2_test[i].reshape((1, 1))
#             })
#             wits_cost_t += wits_cost_tmp
#             u1t += u1tmp
#             u2t += u2tmp
            
#           x0_density = x0_distribution.pdf(x0_test[i])
#           total_density += x0_density
#           scaled_wits_cost = wits_cost_t * x0_density
#           wits_cost_test[0, i] = scaled_wits_cost / test_averaging
#           u1_test[0, i] = u1t / test_averaging
#           u2_test[0, i] = -u2t / test_averaging
#           #x1_test[0, i] = x1t / test_averaging
#       total_cost = np.sum(wits_cost_test) / total_density
#       print('Mean loss over {} points is {}'.format(num_x0_points, total_cost))


[-3.         -2.33333333 -1.66666667 -1.         -0.33333333  0.33333333
  1.          1.66666667  2.33333333  3.        ]
[0.00443185 0.02622189 0.09947714 0.24197072 0.37738323 0.37738323
 0.24197072 0.09947714 0.02622189 0.00443185]
